<a href="https://colab.research.google.com/github/elizabeth-evolkmar/Trabajo-parqueadero-udea/blob/Rama-principal/CODIGO_FINAL_PARQUEADERO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import sys
import time
import platform
import getpass
from datetime import datetime, timedelta

# ------------------------------
# Constantes del sistema
# ------------------------------
MAX_SPACES = 64  # Máximo de espacios disponibles en el parqueadero
HOURLY_RATE = 7000  # Tarifa por hora
QUARTER_HOUR_RATE = 1500  # Tarifa por cada 15 minutos
LOG_FILE = 'event_log.txt'  # Archivo donde se guarda el log de eventos
CSV_FILE = 'report.csv'  # Archivo CSV de reportes

# ------------------------------
# Bases de datos en memoria
# ------------------------------
db_users = {}  # Diccionario de usuarios registrados
parked = {}  # Diccionario de vehículos actualmente parqueados
history = []  # Historial de entradas y salidas
procedures_count = []  # Registro de eventos para auditoría

# ------------------------------
# Usuarios con rol definido
# ------------------------------
admins = {'admin': 'admin123'}
cashiers = {'cajero': 'cajero123'}

# ------------------------------
# Función para registrar eventos
# ------------------------------
def log_event(action):
    now = datetime.now()
    duration = 0
    header = f"User:{getpass.getuser()} | OS:{platform.system()} {platform.release()} | Platform:{platform.platform()}"
    timestamp = now.strftime('%Y-%m-%d %H:%M:%S.%f')
    with open(LOG_FILE, 'a') as f:
        f.write(header + '\n')
        f.write(f"{timestamp}|{action}|{duration:.6f}s\n")
    procedures_count.append(action)

# ------------------------------
# Validaciones básicas
# ------------------------------
def validate_name(name):
    return name.isalpha() and len(name) >= 3

def validate_document(doc):
    return doc.isdigit() and 3 <= len(doc) <= 15

def validate_plate(plate):
    return len(plate) == 6 and plate[:3].isalpha() and plate[3:].isdigit()

# ------------------------------
# Registro de usuario
# ------------------------------
def register_user():
    print("-- Registro de Usuario --")
    name = input("Nombre: ")
    if not validate_name(name):
        print("Error: Nombre inválido.")
        return
    surname = input("Apellido: ")
    if not validate_name(surname):
        print("Error: Apellido inválido.")
        return
    doc = input("Documento: ")
    if not validate_document(doc):
        print("Error: Documento inválido.")
        return
    plate = input("Placa (3 letras y 3 números): ")
    if not validate_plate(plate):
        print("Error: Placa inválida.")
        return
    for u in db_users.values():
        if u['plate'].lower() == plate.lower():
            print("Error: Placa ya registrada.")
            return
    db_users[doc] = {'name': name, 'surname': surname, 'plate': plate.upper()}
    print("Usuario registrado con éxito.")
    log_event('register_user')

# ------------------------------
# Ingreso de vehículo
# ------------------------------
def enter_vehicle():
    print("-- Ingreso de Vehículo --")
    plate = input("Ingrese placa: ").upper()

    user = None
    for datos in db_users.values():
        if datos['plate'].upper() == plate:
            user = datos
            break

    if not user:
        print("Error: Usuario no registrado.")
        return

    if len(parked) >= MAX_SPACES:
        print("Error: No hay espacios disponibles.")
        return

    entry_time = datetime.now()
    parked[plate] = entry_time
    history.append({'plate': plate, 'entry': entry_time, 'exit': None, 'fee': None})
    print(f"Vehículo {plate} ingresado a las {entry_time.strftime('%Y-%m-%d %H:%M:%S')}")
    log_event('enter_vehicle')

# ------------------------------
# Retiro de vehículo
# ------------------------------
def exit_vehicle():
    print("-- Retiro de Vehículo --")
    plate = input("Placa del vehículo: ").upper()
    entry_time = parked.get(plate)
    if not entry_time:
        print("Error: Vehículo no está en el parqueadero.")
        return
    exit_time = datetime.now()
    delta = exit_time - entry_time
    hours = delta.seconds // 3600
    quarters = (delta.seconds % 3600) // 900
    total = max(hours * HOURLY_RATE + quarters * QUARTER_HOUR_RATE, HOURLY_RATE)
    print(f"Total a pagar: {total} pesos")
    for rec in history:
        if rec['plate'] == plate and rec['exit'] is None:
            rec['exit'] = exit_time
            rec['fee'] = total
            break
    del parked[plate]
    log_event('exit_vehicle')

# ------------------------------
# Cierre de caja
# ------------------------------
def cierre_de_caja():
    print("-- Cierre de Caja --")
    total_payments = sum(r['fee'] or 0 for r in history)
    print(f"Total recaudo hasta ahora: {total_payments} pesos")
    log_event('cierre_de_caja')

# ------------------------------
# Exportar historial a CSV
# ------------------------------
def export_csv():
    with open(CSV_FILE, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['plate', 'entry', 'exit', 'fee'])
        writer.writeheader()
        for r in history:
            writer.writerow({
                'plate': r['plate'],
                'entry': r['entry'].strftime('%Y-%m-%d %H:%M:%S'),
                'exit': r['exit'].strftime('%Y-%m-%d %H:%M:%S') if r['exit'] else '',
                'fee': r['fee'] or ''
            })
    print(f"Reporte exportado a {CSV_FILE}")
    log_event('export_csv')

# ------------------------------
# Estadísticas del parqueadero
# ------------------------------
def estadisticas_parqueadero():
    registered = len(set(r['plate'] for r in history))
    retired = sum(1 for r in history if r['exit'] is not None)
    no_retired = sum(1 for r in history if r['exit'] is None)
    total_paid = sum(r['fee'] or 0 for r in history)

    time = [
        (r['exit'] - r['entry']).total_seconds()
        for r in history if r['exit'] and r['entry']
    ]
    average_time = sum(time) / len(time) if time else 0
    average_time_min = round(average_time / 60, 2)

    vehicle_max = vehicle_min = None
    if time:
        vehicle_max = max(
            (r for r in history if r['exit']),
            key=lambda r: (r['exit'] - r['entry']).total_seconds()
        )
        vehicle_min = min(
            (r for r in history if r['exit']),
            key=lambda r: (r['exit'] - r['entry']).total_seconds()
        )

    with open(CSV_FILE, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow([])
        writer.writerow(['---- ESTADÍSTICAS ----'])
        writer.writerow(['Total vehículos registrados (únicos)', registered])
        writer.writerow(['Total vehículos retirados', retired])
        writer.writerow(['Total vehículos sin retirar', no_retired])
        writer.writerow(['Total pagos recibidos', total_paid])
        writer.writerow(['Tiempo promedio de estancia (min)', average_time_min])
        if vehicle_max and vehicle_min:
            writer.writerow(['Vehículo con mayor tiempo parqueado', vehicle_max['plate']])
            writer.writerow(['Vehículo con menor tiempo parqueado', vehicle_min['plate']])

    print("Estadísticas exportadas a report.csv")
    log_event('estadisticas_parqueadero')

# ------------------------------
# Exportar usuarios a CSV
# ------------------------------
def exportar_usuarios():
    with open('users.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['documento', 'nombre', 'apellido', 'placa'])
        writer.writeheader()
        for doc, data in db_users.items():
            writer.writerow({
                'documento': doc,
                'nombre': data['name'],
                'apellido': data['surname'],
                'placa': data['plate']
            })
    print("Usuarios exportados a users.csv")
    log_event('exportar_usuarios')

# ------------------------------
# Login de usuario
# ------------------------------
def login(role_store, role_name):
    user = input(f"Usuario {role_name}: ")
    pwd = input("Contraseña: ")
    if role_store.get(user) == pwd:
        return True
    print("Credenciales incorrectas.")
    return False

# ------------------------------
# Menú del cajero
# ------------------------------
def cashier_menu():
    while True:
        print("\n-- Menú Cajero --")
        print("1. Registrar Usuario")
        print("2. Ingresar Vehículo")
        print("3. Retirar Vehículo")
        print("0. Cerrar sesión")
        c = input("Opción: ")
        if c == '1':
            register_user()
        elif c == '2':
            enter_vehicle()
        elif c == '3':
            exit_vehicle()
        elif c == '0':
            break
        else:
            print("Opción inválida.")

# ------------------------------
# Menú del administrador
# ------------------------------
def admin_menu():
    while True:
        print("\n-- Menú Administrador --")
        print("1. Cierre de Caja")
        print("2. Exportar Reporte CSV")
        print("3. Ver estadísticas generales")
        print("4. Exportar lista de usuarios")
        print("0. Cerrar sesión")
        c = input("Opción: ")
        if c == '1':
            cierre_de_caja()
        elif c == '2':
            export_csv()
        elif c == '3':
            estadisticas_parqueadero()
        elif c == '4':
            exportar_usuarios()
        elif c == '0':
            break
        else:
            print("Opción inválida.")

# ------------------------------
# Función principal
# ------------------------------
def main():
    print(r"""
              ______
           __/  |   \___
          |  _     _  ``-.
     _____| (_)___(_)____|__
    """)
    print("---- Bienvenido parking express ----")
    while True:
        role = input("¿Ingresar como admin o cajero? (admin/cajero/exit): ")
        if role == 'admin':
            if login(admins, 'admin'):
                admin_menu()
        elif role == 'cajero':
            if login(cashiers, 'cajero'):
                cashier_menu()
        elif role == 'exit':
            print("Saliendo...")
            sys.exit()
        else:
            print("Rol inválido.")

if __name__ == '__main__':
    main()


              ______
           __/  |   \___
          |  _     _  ``-.
     _____| (_)___(_)____|__
    
---- Bienvenido parking express ----
¿Ingresar como admin o cajero? (admin/cajero/exit): admin
Usuario admin: admin
Contraseña: admin123

-- Menú Administrador --
1. Cierre de Caja
2. Exportar Reporte CSV
3. Ver estadísticas generales
4. Exportar lista de usuarios
0. Cerrar sesión
Opción: 1
-- Cierre de Caja --
Total recaudo hasta ahora: 0 pesos

-- Menú Administrador --
1. Cierre de Caja
2. Exportar Reporte CSV
3. Ver estadísticas generales
4. Exportar lista de usuarios
0. Cerrar sesión
Opción: 2
Reporte exportado a report.csv

-- Menú Administrador --
1. Cierre de Caja
2. Exportar Reporte CSV
3. Ver estadísticas generales
4. Exportar lista de usuarios
0. Cerrar sesión
Opción: 4
Usuarios exportados a users.csv

-- Menú Administrador --
1. Cierre de Caja
2. Exportar Reporte CSV
3. Ver estadísticas generales
4. Exportar lista de usuarios
0. Cerrar sesión
Opción: 0
¿Ingresar como a

KeyboardInterrupt: Interrupted by user